<a href="https://colab.research.google.com/github/hazem-antar/Egyptian-House-Prices-on-Administrative-2-Level/blob/main/Average_House_Prices_Administrative_2_Level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Gathring Data: Web Scraping

In [3]:
#Importing libraries
from bs4 import BeautifulSoup 
import urllib.request as urllib2
import pandas as pd
import numpy as np
from time import sleep
from tqdm import tqdm
pd.set_option("display.max.columns", None)

In [4]:
#Generating all the URLs of the sale and rent pages (note that odd list indices are for rent pages and evens for sale)
pages = []
pages.append('https://www.olx.com.eg/en/properties/apartments-duplex-for-sale/')
pages.append('https://www.olx.com.eg/en/properties/apartments-duplex-for-rent/')
for i in range(2,501):
    pages.append(pages[0]+"?page="+str(i))
    pages.append(pages[1]+"?page="+str(i))

In [5]:
#Initializing the property_df with your chosen characteristics from OLX
property_df = pd.DataFrame(columns = ["ad_id","type", "price" , "area" , "bedrooms" ,"bathrooms" , "level" ,"furnished" ,"rent", "city" , "region"])

In [ ]:
#Designing the scrapper!

hold_dict={}    #A dictionary that will hold the values of features for each single house untile regiesterd in property_df

opener = urllib2.build_opener()   #Creating an object of a URL opener
opener.addheaders = [('User-agent', 'Mozilla/5.0')] #Setting virtual headers to the object as to be accessing OLX like a normal user from a website not from an automated script.
for i in tqdm(range(0,1000)):   #Iterate through the number of total ads pages we have
    
    with opener.open(pages[i]) as page:   #Accssing the ads page URL in our generated list of URLs (requesting its HTML code)
        page_html = page.read().decode('utf-8')   #Read and decode the incoming HTML to a string
        page_soup = BeautifulSoup(page_html, 'html.parser')   #Parse and convert the string into a tree-like structure of parent and children tags
        ads = page_soup.find_all('a',attrs = {'class','ads__item__ad--title'})    #Look for any class-type tag in the tree with the name 'ads__item__ad--title', which represents a block of ad.
        
        for ad in ads:  #Iterate through the acquired blocks of ads
            for n in property_df.columns: hold_dict[n] = np.nan   #Reset the hold_dict to have NaN for all keys in the property_df
            if i%2==0 :hold_dict['rent'] = 'no'  #if i is even the ad was from a sale page
            else:hold_dict['rent'] = 'yes'  #if i is odd the ad was from a rent page
            find = ad["href"].index('/ad/')   #Extracting the ad page URL
            full_ad_url = ad["href"][0:find] + "/en" + ad["href"][find:]   #Generatinng the English version as it automatically makes an arabic one
           
            with opener.open(full_ad_url) as ad_page:   #Accssing the specific ad page URL (requesting its HTML code)
                ad_page_html = ad_page.read().decode('utf-8')
                ad_page_soup = BeautifulSoup(ad_page_html, 'html.parser')
                #Specifing some wanted and unwanted words in the ad title and body-text so we can filter out the unwanted silly ads:
                wanted_words = ["شقة" , "شقه" ,"شقتك" ,"شقق", "بيت" , "بيتك" , "منزل" , "دوبلكس" , "ستوديو" ,"استوديو" , "بنتهاوس" , "وحدة" , "وحده" ,"وحدات" , "ارضي" , "ارضى" , "apartment" , "penthouse" , "duplex" , "studio"]
                unwanted_words = ["ايجار قديم" , "إيجار قديم", "قانون قديم" , "أيجار قديم"]
                wanted_exists , unwanted_exists = False , False

                title = ad_page_soup.find('h1',attrs = {'class','brkword lheight28'}) #Find the tag holding the ad-title
                if title == None: continue  #if the ad has no title skip the ad, as it is mostly a spam ad.
                else: 
                  for word in wanted_words:
                    if word in title.get_text().lower():
                      wanted_exists = True
                      break
                  for word in unwanted_words:
                    if word in title.get_text().lower():
                      unwanted_exists = True
                      break

                body_text = ad_page_soup.find('p',attrs = {'class','pding10 lheight20 large'})    #Find the tag holding the body-text
                if body_text != None:  #if body text exists, it's necessary to prevent the below lines from throwing an error
                  for word in unwanted_words:
                    if word in body_text.get_text().lower():
                      unwanted_exists = True
                      break

                if (wanted_exists == False) or (unwanted_exists == True): continue    # if the ad contains an unwanted word or does not contain a wanted word, skip it.

                id = ad_page_soup.find('span',attrs = {'class','rel inlblk'})   #Find the tag holding the ad-id
                if id == None: continue   #if no ad-id, skip the ad
                elif len(id.get_text()) == 0: continue    #if there is ad-id but it is empty, skip the ad
                hold_dict['ad_id'] = id.get_text().strip()    #save in hold_dict for later usage

                price = ad_page_soup.find('strong',attrs = {'class','xxxx-large margintop7 block not-arranged'})     #Find the tag holding the ad-price
                if price == None: continue
                elif len(price.get_text()) == 0: continue
                price_value = price.get_text().replace(",","").replace("EGP" ,"").strip()   #convert from the form "500,000EGP/n" to "500000"
                suspicious_value = True   #Some ads puts strange prices like "1111111111111", skip them.
                for k in price_value:
                  if k != price_value[0] : suspicious_value = False
                if suspicious_value == True: continue
                hold_dict['price'] = price.get_text().replace(",","").replace("EGP" ,"").strip()

                location = ad_page_soup.find('strong',attrs = {'class','c2b small'}).get_text()   #Find the tag holding the ad-location
                if "،" in location:
                    location = location.split("،")
                elif "," in location:
                    location = location.split(",")
                hold_dict['city'] , hold_dict['region'] = location[1].strip().lower() , location[0].strip().lower()

                data_table = ad_page_soup.find('table', attrs={'class':'details fixed marginbott20 margintop5 full'}).find_all('tr')  #select the table in the ad page, it contains some features like the area and number of bedrooms and a few others.
                for raw in data_table:
                    features = raw.find_all('td') #each raw in the table contain some featues and their crossponding values, select them.
                    for feature in features:
                        if feature == None: continue    #check if feature content in not null
                        elif len(feature.get_text()) == 0: continue
                        if feature.find('th') == None: continue  
                        elif len(feature.find('th').get_text()) == 0: continue
                        key = feature.find('th').get_text().strip().lower()
                        if key == 'area (m²)': key = 'area'
                        if(key in property_df.columns):   #if the feature in our property_df coulmns add the feature and value to the hold_dict
                            if feature.find('strong') == None: continue  #check if feature value in not null
                            elif len(feature.find('strong').get_text()) == 0: continue
                            hold_dict[key] = feature.find('strong').get_text().split(",")[0].strip().lower()

            df_temp = pd.DataFrame({"ad_id":[hold_dict["ad_id"]],"type":[hold_dict["type"]], "price":[hold_dict["price"]] , "area":[hold_dict["area"]] , "bedrooms":[hold_dict["bedrooms"]] ,"bathrooms":[hold_dict["bathrooms"]] , "level":[hold_dict["level"]] ,"furnished":[hold_dict["furnished"]] ,"rent":[hold_dict["rent"]], "city":[hold_dict["city"]] , "region":[hold_dict["region"]]})
            property_df = property_df.append(df_temp,ignore_index = True)  #append the data record to original property dataframe
            sleep(0.25)
    sleep(0.25)

# Inspecting and Cleaning the Data

In [ ]:
#drop duplicate records by house id on OLX
property_df.drop_duplicates(subset="ad_id", keep = 'first', inplace = True)
property_df = property_df.reset_index(drop=True)

In [ ]:
property_df

,ad_id,type,price,area,bedrooms,bathrooms,level,furnished,rent,city,region
0,181762848,apartment,1280000.0,100.0,2,1,3,no,no,giza,sheikh zayed
1,181683147,apartment,1270000.0,162.0,3,3,4,NaN,no,cairo,new cairo - el tagamoa
2,172174108,apartment,858000.0,156.0,3,3,1,no,no,cairo,new heliopolis
3,180487915,apartment,125000.0,135.0,2,2,5,NaN,no,cairo,obour city
4,181630873,apartment,2000000.0,173.0,3,3,7,no,no,cairo,new capital city
...,...,...,...,...,...,...,...,...,...,...,...
26448,178804236,apartment,19999.0,244.0,3,3,ground,NaN,yes,cairo,new cairo - el tagamoa
26449,179744558,apartment,15000.0,250.0,3,1,2,yes,yes,cairo,sheraton
26450,179603444,apartment,4200.0,96.0,2,2,5,no,yes,cairo,madinaty
26451,179306157,apartment,17000.0,157.0,3,3,4,NaN,yes,cairo,new cairo - el tagamoa


In [ ]:
property_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26453 entries, 0 to 26452
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ad_id      26453 non-null  int64  
 1   type       26446 non-null  object 
 2   price      26453 non-null  float64
 3   area       26453 non-null  float64
 4   bedrooms   26448 non-null  object 
 5   bathrooms  26448 non-null  object 
 6   level      26446 non-null  object 
 7   furnished  19542 non-null  object 
 8   rent       26453 non-null  object 
 9   city       26453 non-null  object 
 10  region     26453 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 2.2+ MB


In [ ]:
property_df.isna().sum()

ad_id           0
type            7
price           0
area            0
bedrooms        5
bathrooms       5
level           7
furnished    6911
rent            0
city            0
region          0
dtype: int64

In [ ]:
property_df['city'].value_counts()

cairo             14548
giza               5644
alexandria         3004
dakahlia            502
sharqia             329
gharbia             313
qalyubia            251
port said           240
red sea             197
asyut               181
damietta            172
ismailia            166
matruh              164
suez                135
monufia             131
minya                83
beheira              74
fayoum               53
aswan                51
beni suef            45
kafr al-sheikh       44
sohag                38
qena                 38
south sinai          29
luxor                20
new valley            1
Name: city, dtype: int64

In [ ]:
property_df['level'].value_counts()

2          4899
3          4268
1          3885
ground     3697
4          2996
5          2197
6          1064
7           792
10+         729
8           585
9           556
10          526
highest     252
Name: level, dtype: int64

In [ ]:
property_df['region'].unique()

array(['sheikh zayed', 'new cairo - el tagamoa', 'new heliopolis',
       'obour city', 'new capital city', 'smoha', 'new nozha',
       '6th of october', 'hadayek 6th of october', 'ain shams', 'bacchus',
       'katameya', 'madinaty', 'shorouk city', 'hadayek al-ahram',
       'mokattam', 'badr city', 'nasr city', 'roushdy', 'mohandessin',
       'glim', 'agami', 'zahraa al maadi', 'maadi', 'haram', 'fleming',
       'gesr al suez', 'helmeyat el zaytoun', 'alamein', 'mansura',
       'hurghada', 'agouza', 'shubra al-khaimah', 'downtown cairo',
       'miami', '10th of ramadan', 'rehab city', 'dar al-salaam',
       'dhahria', 'shebin al-koum', 'faisal', 'suez district',
       'kafr abdo', 'basateen', 'matareya', 'sheraton', 'mandara',
       'sidi beshr', 'tookh', 'giza district', 'seyouf', 'helwan',
       'mahalla al-kobra', 'marsa matrouh', 'nakheel', 'dokki', 'labban',
       'tanta', 'bahtim', 'ismailia city', 'al ibrahimiyyah', 'khosous',
       'north coast', 'laurent', 'san s

In [ ]:
#Converting string data into numerical equivalent
for i in range(len(property_df)):
  if property_df.loc[i ,'level'] == 'ground':
    property_df.loc[i ,'level'] = 0
  elif property_df.loc[i ,'level'] == '10+' :
    property_df.loc[i ,'level'] = 12
  elif property_df.loc[i ,'level'] == 'highest':
    property_df.loc[i ,'level'] = 12  
  if property_df.loc[i ,'bedrooms'] == '10+':
    property_df.loc[i ,'bedrooms'] = 12
  if property_df.loc[i ,'bathrooms'] == '10+':
    property_df.loc[i ,'bathrooms'] = 12
  if property_df.loc[i ,'rent'] == 'no':
    property_df.loc[i ,'rent'] = 0
  elif property_df.loc[i ,'rent'] == 'yes':
    property_df.loc[i ,'rent'] = 1
  if property_df.loc[i ,'furnished'] == 'no':
    property_df.loc[i ,'furnished'] = 0
  elif property_df.loc[i ,'furnished'] == 'yes':
    property_df.loc[i ,'furnished'] = 1
  if ' district' in property_df.loc[i ,'region']: 
    property_df.loc[i ,'region'] = property_df.loc[i ,'region'].replace(' district','')
  elif property_df.loc[i ,'region'] == 'new cairo - el tagamoa':
    property_df.loc[i ,'region'] = 'el tagamoa'
  property_df.loc[i ,'area'] = int(str(property_df.loc[i ,'area']).split(".")[0])

In [ ]:
# Building three classifiers to predict the missing values of the house number of bedrooms, bathrooms, and whether the home is furnished or not; Taking into account the house price, area, and whether it's for rent or sale
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

model_bed = KNeighborsClassifier(n_neighbors=len(property_df.dropna(subset=["bedrooms"])["bedrooms"].unique()))
X_train_bed, X_test_bed, y_train_bed, y_test_bed = train_test_split(property_df.dropna(subset=["bedrooms"])[["price","area","rent"]].astype(int), property_df.dropna(subset=["bedrooms"])["bedrooms"].astype(int), test_size=0.2)
model_bath = KNeighborsClassifier(n_neighbors=len(property_df.dropna(subset=["bathrooms"])["bathrooms"].unique()))
X_train_bath, X_test_bath, y_train_bath, y_test_bath = train_test_split(property_df.dropna(subset=["bathrooms"])[["price","area","rent"]].astype(int), property_df.dropna(subset=["bathrooms"])["bathrooms"].astype(int), test_size=0.2)
model_level = KNeighborsClassifier(n_neighbors=len(property_df.dropna(subset=["level"])["level"].unique()))
X_train_level, X_test_level, y_train_level, y_test_level = train_test_split(property_df.dropna(subset=["level"])[["price","area","rent"]].astype(int), property_df.dropna(subset=["level"])["level"].astype(int), test_size=0.2)
model_fur = KNeighborsClassifier(n_neighbors=len(property_df.dropna(subset=["furnished"])["furnished"].unique()))
X_train_fur, X_test_fur, y_train_fur, y_test_fur = train_test_split(property_df.dropna(subset=["furnished"])[["price","area","rent"]], property_df.dropna(subset=["furnished"])["furnished"], test_size=0.2)

model_bed.fit(X_train_bed, y_train_bed.astype('int'))
model_bath.fit(X_train_bath, y_train_bath.astype('int'))
model_level.fit(X_train_level, y_train_level.astype('int'))
model_fur.fit(X_train_fur, y_train_fur.astype('int'))
print(model_bed.score(X_test_bed, y_test_bed.astype('int')) , model_bath.score(X_test_bath, y_test_bath.astype('int')) , model_level.score(X_test_level, y_test_level.astype('int')) , model_fur.score(X_test_fur, y_test_fur.astype('int')))


0.7109640831758034 0.6850661625708885 0.20245746691871455 0.8083908928114607


In [ ]:
# Filling missing values with the appropriate values. Note that the model for predicting the level did not perform well so we will take the mean value.
property_df['type'].fillna(property_df.dropna(subset=["type"])['type'].mode()[0],inplace=True)
property_df['level'].fillna(int(property_df.dropna(subset=["level"])['level'].astype(int).mean()),inplace=True)
for i in range(len(property_df)):
  if pd.isnull(property_df.loc[i , 'bathrooms']):
    property_df.loc[i , 'bathrooms'] = model_bath.predict([property_df.loc[i ,["price","area","rent"]]])[0]
  if pd.isnull(property_df.loc[i , 'bedrooms']):
    property_df.loc[i , 'bedrooms'] = model_bed.predict([property_df.loc[i ,["price","area","rent"]]])[0]
  if pd.isnull(property_df.loc[i , 'furnished']):
    property_df.loc[i , 'furnished'] = model_fur.predict([property_df.loc[i ,["price","area","rent"]]])[0]

In [ ]:
# Making sure that we have numeric columns for later usage
property_df['level'] = property_df['level'].astype(int)
property_df['bathrooms'] = property_df['bathrooms'].astype(int)
property_df['bedrooms'] = property_df['bedrooms'].astype(int)
property_df['area'] = property_df['area'].astype(int)
property_df['rent'] = property_df['rent'].astype(int)
property_df['furnished'] = property_df['furnished'].astype(int)

In [ ]:
property_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26453 entries, 0 to 26452
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ad_id      26453 non-null  int64  
 1   type       26453 non-null  object 
 2   price      26453 non-null  float64
 3   area       26453 non-null  int64  
 4   bedrooms   26453 non-null  int64  
 5   bathrooms  26453 non-null  int64  
 6   level      26453 non-null  int64  
 7   furnished  26453 non-null  int64  
 8   rent       26453 non-null  int64  
 9   city       26453 non-null  object 
 10  region     26453 non-null  object 
dtypes: float64(1), int64(7), object(3)
memory usage: 2.2+ MB


In [ ]:
# Dropping nonsensible house areas that are most likely miswritten
property_df.drop(property_df[property_df["area"] > 600].index , inplace = True)
property_df.drop(property_df[property_df["area"] < 20].index , inplace = True)
property_df = property_df.reset_index(drop=True)

In [ ]:
# Creating a column for area_category that will be used in categorizing houses into groups and reject outliers from each group
property_df["area_category"] = np.ceil(property_df["area"]/100)
price_by_group = property_df[["city" , "region" , "price" , 'type' , 'rent' , "furnished" , "area_category"]].groupby(["city" , "region" , 'type' , 'rent' , 'area_category' ,"furnished"])

In [ ]:
# Calculating ZScore for each house based only on its specific group and dropping any house that is more than 3 Standard Deviations

from scipy import stats

for i in price_by_group:
  if len(i[1]["price"]) < 2 :continue
  scores = np.abs(stats.zscore(i[1]["price"]))
  ind = i[1]["price"].index
  for j in range(len(i[1])):
    if scores[j] > 3:
       property_df.drop([ind[j]] ,axis=0, inplace = True)
property_df.drop(["area_category"], axis=1 , inplace= True)
property_df = property_df.reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:2419: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


#Determining House Locations on Administrative 2 Level

In [ ]:
!pip install --upgrade geopandas
!pip install --upgrade shapely

In [ ]:
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, Polygon
import fiona
import requests

In [ ]:
#Using Geoapify API to get the geographical location of an address we can gain the a point referring to the center of the region where the house is located

url = 'https://api.geoapify.com/v1/geocode/search'
property_df["region_center"] = np.nan
address_to_center={}
for i in tqdm(range(len(property_df))):
  full_address = property_df.loc[i, "region"] +", " + property_df.loc[i, "city"] +", Egypt"
  if(full_address not in address_to_center):
    params = dict( text = full_address, apiKey = '375574c2c64049809f237d8f2880c9fa')
    resp = requests.get(url=url, params=params).json()
    center = Point(resp['features'][0]['properties']['lon'],resp['features'][0]['properties']['lat'])
    property_df.loc[i,"region_center"] = str(center)
    address_to_center[full_address] = str(center)
  else:
    property_df.loc[i,"region_center"] = address_to_center[full_address]

100%|██████████| 25424/25424 [03:46<00:00, 112.46it/s]


In [ ]:
property_df

,ad_id,type,price,area,bedrooms,bathrooms,level,furnished,rent,city,region,region_center
0,181762848,apartment,1280000.0,100,2,1,3,0,0,giza,sheikh zayed,POINT (30.9832235 30.048347)
1,181683147,apartment,1270000.0,162,3,3,4,0,0,cairo,el tagamoa,POINT (31.4267324 30.0023841)
2,172174108,apartment,858000.0,156,3,3,1,0,0,cairo,new heliopolis,POINT (31.6925603 30.1180968)
3,180487915,apartment,125000.0,135,2,2,5,0,0,cairo,obour city,POINT (31.4785241 30.163356)
4,181630873,apartment,2000000.0,173,3,3,7,0,0,cairo,new capital city,POINT (31.4752658 30.0288198)
...,...,...,...,...,...,...,...,...,...,...,...,...
25419,178804236,apartment,19999.0,244,3,3,0,1,1,cairo,el tagamoa,POINT (31.4267324 30.0023841)
25420,179744558,apartment,15000.0,250,3,1,2,1,1,cairo,sheraton,POINT (33.850244 27.199106)
25421,179603444,apartment,4200.0,96,2,2,5,0,1,cairo,madinaty,POINT (31.637687 30.084327)
25422,179306157,apartment,17000.0,157,3,3,4,1,1,cairo,el tagamoa,POINT (31.4267324 30.0023841)


In [ ]:
property_df.isna().sum()

ad_id            0
type             0
price            0
area             0
bedrooms         0
bathrooms        0
level            0
furnished        0
rent             0
city             0
region           0
region_center    0
dtype: int64

Mapping the region center point returned from the API to a unique administrative level 2

In [ ]:
import io
from google.colab import files
uploaded_lev2 = files.upload()

Saving egy_admbnda_adm2_capmas_20170421.cpg to egy_admbnda_adm2_capmas_20170421.cpg
Saving egy_admbnda_adm2_capmas_20170421.dbf to egy_admbnda_adm2_capmas_20170421.dbf
Saving egy_admbnda_adm2_capmas_20170421.prj to egy_admbnda_adm2_capmas_20170421.prj
Saving egy_admbnda_adm2_capmas_20170421.shp to egy_admbnda_adm2_capmas_20170421.shp
Saving egy_admbnda_adm2_capmas_20170421.shp.xml to egy_admbnda_adm2_capmas_20170421.shp.xml
Saving egy_admbnda_adm2_capmas_20170421.shx to egy_admbnda_adm2_capmas_20170421.shx


In [ ]:
marakez_lev2 = gpd.read_file("egy_admbnda_adm2_capmas_20170421.shp")[["ADM0_EN",	"ADM0_AR" , "ADM1_EN",	"ADM1_AR", "ADM2_EN"	,"ADM2_AR", "geometry"]]
marakez_lev2

,ADM0_EN,ADM0_AR,ADM1_EN,ADM1_AR,ADM2_EN,ADM2_AR,geometry
0,Egypt,مِصر,Sharkia,الشرقية\n,10 Ramadan 1,قسم اول مدينة العاشر من رمض,"POLYGON ((31.70323 30.21166, 31.70354 30.21452..."
1,Egypt,مِصر,Sharkia,الشرقية\n,10 Ramadan 2,قسم ثان مدينه العاشر من رمض,"POLYGON ((31.72752 30.25126, 31.72671 30.25009..."
2,Egypt,مِصر,Cairo,القاهرة,15 Mayu,قسم 15 مايو,"POLYGON ((31.37739 29.78892, 31.36398 29.78987..."
3,Egypt,مِصر,Giza,الجيزة,6 October-1,قسم أول 6 أكتوبر,"POLYGON ((30.98266 30.00870, 30.98959 30.00103..."
4,Egypt,مِصر,Giza,الجيزة,6 October-2,قسم ثان 6 أكتوبر,"POLYGON ((30.95400 29.95111, 30.95393 29.95104..."
...,...,...,...,...,...,...,...
360,Egypt,مِصر,Luxor,مدينة الأقصر,Zemam Out,خارج الزمام,"POLYGON ((32.49782 25.54791, 32.49789 25.54802..."
361,Egypt,مِصر,Assiut,أسيوط,Zemam Out,صحراء أسيوط,"MULTIPOLYGON (((32.56866 27.85326, 32.56885 27..."
362,Egypt,مِصر,Suhag,سوهاج,Zemam Out,صحراء سوهاج,"MULTIPOLYGON (((32.74119 26.68548, 32.28002 26..."
363,Egypt,مِصر,Qena,قنا,Zemam Out,صحراء قنا,"MULTIPOLYGON (((32.93116 26.69319, 32.93297 26..."


In [ ]:
# Mapping the region center point returned from the API to a unique administrative level 2
import shapely.wkt
property_df["region_bbox"] = np.nan
property_df["ADM2_AR"] = np.nan
center_to_level2={}
for i in tqdm(range(len(property_df))):
  cent = shapely.wkt.loads(property_df.loc[i,["region_center"]][0])
  if str(cent) in center_to_level2:
    property_df.loc[i,["region_bbox"]] = center_to_level2[str(cent)][0]
    property_df.loc[i,["ADM2_AR"]] = center_to_level2[str(cent)][1]
  else:
    for j in range(len(marakez_lev2)):
      shp = marakez_lev2.loc[j,["geometry"]]
      if cent.within(shp[0]):
        property_df.loc[i,["region_bbox"]] = str(shp[0])
        property_df.loc[i,["ADM2_AR"]] = marakez_lev2.loc[j,["ADM2_AR"]]
        center_to_level2[str(cent)] = (str(shp[0]) , marakez_lev2.loc[j,["ADM2_AR"]])
        break

100%|██████████| 25424/25424 [02:56<00:00, 144.15it/s]


In [ ]:
property_df

,ad_id,type,price,area,bedrooms,bathrooms,level,furnished,rent,city,region,region_center,region_bbox,ADM2_AR
0,181762848,apartment,1280000.0,100,2,1,3,0,0,giza,sheikh zayed,POINT (30.9832235 30.048347),"POLYGON ((30.98266033100003 30.00870261500006,...",قسم الشيخ زايد
1,181683147,apartment,1270000.0,162,3,3,4,0,0,cairo,el tagamoa,POINT (31.4267324 30.0023841),"POLYGON ((31.69729902400007 30.03950206300004,...",قسم أول القاهرة الجديدة
2,172174108,apartment,858000.0,156,3,3,1,0,0,cairo,new heliopolis,POINT (31.6925603 30.1180968),"POLYGON ((31.72444992400006 30.11064236300007,...",قسم بدر
3,180487915,apartment,125000.0,135,2,2,5,0,0,cairo,obour city,POINT (31.4785241 30.163356),"POLYGON ((31.65415030200006 30.11361469200006,...",قسم الشروق
4,181630873,apartment,2000000.0,173,3,3,7,0,0,cairo,new capital city,POINT (31.4752658 30.0288198),"POLYGON ((31.69729902400007 30.03950206300004,...",قسم أول القاهرة الجديدة
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25419,178804236,apartment,19999.0,244,3,3,0,1,1,cairo,el tagamoa,POINT (31.4267324 30.0023841),"POLYGON ((31.69729902400007 30.03950206300004,...",قسم أول القاهرة الجديدة
25420,179744558,apartment,15000.0,250,3,1,2,1,1,cairo,sheraton,POINT (33.850244 27.199106),"POLYGON ((33.67930977900005 27.37594298900007,...",قسم أول الغردقة
25421,179603444,apartment,4200.0,96,2,2,5,0,1,cairo,madinaty,POINT (31.637687 30.084327),"POLYGON ((31.65415030200006 30.11361469200006,...",قسم ثان القاهرة الجديدة
25422,179306157,apartment,17000.0,157,3,3,4,1,1,cairo,el tagamoa,POINT (31.4267324 30.0023841),"POLYGON ((31.69729902400007 30.03950206300004,...",قسم أول القاهرة الجديدة


In [ ]:
property_df.isna().sum()

ad_id              0
type               0
price              0
area               0
bedrooms           0
bathrooms          0
level              0
furnished          0
rent               0
city               0
region             0
region_center      0
region_bbox      204
ADM2_AR          204
dtype: int64

In [ ]:
# Reigons that were not found in any admin2 means that they were mislocated by the API and that is clear looking at the LAT and LON of their centers. We will have to get them manually from Google Earth
property_df.drop_duplicates(subset=["city","region"])[property_df.drop_duplicates(subset=["city","region"])['region_bbox'].isnull()]

,ad_id,type,price,area,bedrooms,bathrooms,level,furnished,rent,city,region,region_center,region_bbox,ADM2_AR
141,181784862,apartment,470000.0,126,2,1,5,1,0,cairo,dar al-salaam,POINT (37.689261 -6.803131),NaN,NaN
227,177869747,apartment,200.0,70,1,1,1,1,1,matruh,marsa matrouh,POINT (27.2396987 31.3562414),NaN,NaN
546,177770309,apartment,1000000.0,125,2,1,1,0,0,cairo,boulaq,POINT (31.22892552926209 30.0630926),NaN,NaN
865,181336853,apartment,850000.0,280,3,2,0,0,0,fayoum,yusuf al-sadiq,POINT (47.863797 40.996438),NaN,NaN
993,181762352,apartment,8000.0,125,2,1,5,0,1,alexandria,camp caesar,POINT (-71.518584 42.260486),NaN,NaN
2525,180321445,apartment,400.0,67,2,1,5,0,1,asyut,abu teeg,POINT (44.18694 33.30703),NaN,NaN
4840,181511551,apartment,700.0,75,2,1,2,0,1,sharqia,new salhia,POINT (-86.15330899999999 34.077596),NaN,NaN
7758,181302349,apartment,250.0,150,3,1,2,1,1,kafr al-sheikh,baltim,POINT (31.0915988 31.5383944),NaN,NaN
9997,181132253,apartment,600.0,60,2,1,0,0,1,beheira,hosh essa,POINT (-73.940707 40.722682),NaN,NaN
11692,168622559,apartment,300000.0,110,3,1,1,0,0,monufia,ashmon,POINT (-71.06307 42.283679),NaN,NaN


In [ ]:
uploaded_missing = files.upload()

Saving abu hammad,sharqia.kml to abu hammad,sharqia (2).kml
Saving abu teeg,asyut.kml to abu teeg,asyut (2).kml
Saving almazah,cairo.kml to almazah,cairo (2).kml
Saving ashmon,monufia.kml to ashmon,monufia (2).kml
Saving baltim,kafr al-sheikh.kml to baltim,kafr al-sheikh (2).kml
Saving bashtil,giza.kml to bashtil,giza (2).kml
Saving boulaq dakrour,giza.kml to boulaq dakrour,giza (2).kml
Saving boulaq,cairo.kml to boulaq,cairo (2).kml
Saving camp caesar,alexandria.kml to camp caesar,alexandria (2).kml
Saving dar al-salaam,cairo.kml to dar al-salaam,cairo (2).kml
Saving etay al-barud,beheira.kml to etay al-barud,beheira (2).kml
Saving hihya,sharqia.kml to hihya,sharqia (2).kml
Saving hosh essa,beheira.kml to hosh essa,beheira (2).kml
Saving kantara east,ismailia.kml to kantara east,ismailia (2).kml
Saving kantara west,ismailia.kml to kantara west,ismailia (2).kml
Saving manfalut,asyut.kml to manfalut,asyut (2).kml
Saving markaz el matareya,dakahlia.kml to markaz el matareya,dakahlia (2).

In [ ]:
# replacing misidentified location with their actual locations and dropping the reign center column as we no longer need it
fiona.drvsupport.supported_drivers['kml'] = 'rw' 
fiona.drvsupport.supported_drivers['KML'] = 'rw'

missingAdress_to_Actualshape = {}
empty = property_df.drop_duplicates(subset=["city","region"])[property_df.drop_duplicates(subset=["city","region"])['region_bbox'].isnull()]
empty = empty.reset_index(drop=True)
for i in range(len(empty)):
  missing_ad = str(empty.loc[i,["city" , "region"]]["region"]) + "," + str(empty.loc[i,["city" , "region"]]["city"])
  shape = str(gpd.read_file(io.BytesIO(uploaded_missing[str(empty.loc[i,["city" , "region"]]["region"]) + "," + str(empty.loc[i,["city" , "region"]]["city"]) +'.kml']),driver="KML")["geometry"][0])
  missingAdress_to_Actualshape[missing_ad] = shape
for i in tqdm(range(len(property_df))):
  if pd.isnull(property_df.loc[i, "region_bbox"]): 
    found_shape = shapely.wkt.loads(missingAdress_to_Actualshape[property_df.loc[i,"region"] +"," + property_df.loc[i,"city"]])
    for j in range(len(marakez_lev2)):
      markz = marakez_lev2.loc[j,'geometry']
      if found_shape.centroid.within(markz):
        property_df.loc[i, "region_bbox"] = str(markz)
        property_df.loc[i, "ADM2_AR"] = marakez_lev2.loc[j, "ADM2_AR"]
        break

property_df.drop(["region_center"] , axis = 1 , inplace= True)

100%|██████████| 25424/25424 [00:03<00:00, 6512.48it/s]


In [ ]:
property_df.isna().sum()

ad_id          0
type           0
price          0
area           0
bedrooms       0
bathrooms      0
level          0
furnished      0
rent           0
city           0
region         0
region_bbox    0
ADM2_AR        0
dtype: int64

#Calculations of average house prices

In [ ]:
# Calculating price per meter for each house
property_df["price/m"] = (property_df["price"].astype(np.float64) / property_df["area"].astype(np.float64))

In [ ]:
# Dropping unlogical data that passed ZScore filtrations because they were alone in their groups. The criteria of price per meter to drop were calculated based on detailed observation of the normal prices of the most expensive houses.
for i in range(len(property_df)):
  if property_df.loc[i,["price/m"]][0] >25500 and property_df.loc[i,["rent"]][0] == 0 :
    property_df.drop([i], axis= 0, inplace = True)
  elif property_df.loc[i,["price/m"]][0] >800 and property_df.loc[i,["rent"]][0] == 1 :
    property_df.drop([i], axis= 0, inplace = True)
property_df = property_df.reset_index(drop=True)

In [ ]:
# Groping houses based on their level 2 and if they are for sale or rent
price_by_location = property_df[['ADM2_AR', 'region_bbox' ,'rent' ,"price/m"]].groupby(['region_bbox', 'rent'])

In [ ]:
# Creating a dictionary that takes level 2 and return both the means of both (sale/m) and (rent/m), and another one that counts the number of houses used in calculating the means.
sale_rent_means = {}
house_counts = {}

for j in price_by_location:
  poly = j[0][0]
  rent = j[1]["rent"].values[0]
  mean_price = j[1]["price/m"].mean()
  count_houses = j[1]["price/m"].count()
  if poly not in sale_rent_means:
    sale_rent_means[poly] = [(rent , mean_price)]
    house_counts[poly] = [(rent ,count_houses)]
  else:
    sale_rent_means[poly] += [(rent , mean_price)]
    house_counts[poly] += [(rent ,count_houses)]

In [ ]:
# Creating new columns  (rent/m) and (sale/m) and filling them from the dictionary
marakez_lev2["rent/m"]=np.nan
marakez_lev2["sale/m"]=np.nan
for i in range(len(marakez_lev2)):
  if str(marakez_lev2.loc[i,'geometry']) in sale_rent_means:
    for j in sale_rent_means[str(marakez_lev2.loc[i,'geometry'])]:
      if j[0] == 0:
          marakez_lev2.loc[i,'sale/m'] = j[1]
      else:
          marakez_lev2.loc[i,'rent/m'] = j[1]

In [ ]:
marakez_lev2.isna().sum()

ADM0_EN       0
ADM0_AR       0
ADM1_EN       0
ADM1_AR       0
ADM2_EN       0
ADM2_AR       0
geometry      0
rent/m      215
sale/m      215
dtype: int64

In [ ]:
# Creating new coulmns (house_counts_rent) and (house_counts_sale) and filling them from the house_counts dictionary
marakez_lev2["house_counts_rent"] = np.nan
marakez_lev2["house_counts_sale"] = np.nan
for i in range(len(marakez_lev2)):
  if str(marakez_lev2.loc[i,'geometry']) in house_counts:
    for j in house_counts[str(marakez_lev2.loc[i,'geometry'])]:
      if j[0] == 0:
        marakez_lev2.loc[i,'house_counts_sale'] = int(j[1])
      else:
        marakez_lev2.loc[i,'house_counts_rent'] = int(j[1])

In [ ]:
marakez_lev2

,ADM0_EN,ADM0_AR,ADM1_EN,ADM1_AR,ADM2_EN,ADM2_AR,geometry,rent/m,sale/m,house_counts_rent,house_counts_sale
0,Egypt,مِصر,Sharkia,الشرقية\n,10 Ramadan 1,قسم اول مدينة العاشر من رمض,"POLYGON ((31.70323 30.21166, 31.70354 30.21452...",NaN,NaN,NaN,NaN
1,Egypt,مِصر,Sharkia,الشرقية\n,10 Ramadan 2,قسم ثان مدينه العاشر من رمض,"POLYGON ((31.72752 30.25126, 31.72671 30.25009...",NaN,NaN,NaN,NaN
2,Egypt,مِصر,Cairo,القاهرة,15 Mayu,قسم 15 مايو,"POLYGON ((31.37739 29.78892, 31.36398 29.78987...",17.285814,3807.235259,19.0,30.0
3,Egypt,مِصر,Giza,الجيزة,6 October-1,قسم أول 6 أكتوبر,"POLYGON ((30.98266 30.00870, 30.98959 30.00103...",39.461182,6994.312653,618.0,899.0
4,Egypt,مِصر,Giza,الجيزة,6 October-2,قسم ثان 6 أكتوبر,"POLYGON ((30.95400 29.95111, 30.95393 29.95104...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
360,Egypt,مِصر,Luxor,مدينة الأقصر,Zemam Out,خارج الزمام,"POLYGON ((32.49782 25.54791, 32.49789 25.54802...",NaN,NaN,NaN,NaN
361,Egypt,مِصر,Assiut,أسيوط,Zemam Out,صحراء أسيوط,"MULTIPOLYGON (((32.56866 27.85326, 32.56885 27...",28.359280,5381.562882,6.0,4.0
362,Egypt,مِصر,Suhag,سوهاج,Zemam Out,صحراء سوهاج,"MULTIPOLYGON (((32.74119 26.68548, 32.28002 26...",NaN,NaN,NaN,NaN
363,Egypt,مِصر,Qena,قنا,Zemam Out,صحراء قنا,"MULTIPOLYGON (((32.93116 26.69319, 32.93297 26...",NaN,NaN,NaN,NaN


In [ ]:
# Creating a normalized column for rent/m and sale/m and creating an index column that is the average of both
marakez_lev2["rent/m(norm)"] = ( marakez_lev2["rent/m"] - (marakez_lev2["rent/m"]).min() ) / ( (marakez_lev2["rent/m"]).max() - (marakez_lev2["rent/m"]).min() )
marakez_lev2["sale/m(norm)"] = ( marakez_lev2["sale/m"] - (marakez_lev2["sale/m"]).min() ) / ( (marakez_lev2["sale/m"]).max() - (marakez_lev2["sale/m"]).min() )
marakez_lev2["index"] = (marakez_lev2["rent/m(norm)"] + marakez_lev2["sale/m(norm)"])/2

In [ ]:
marakez_lev2

,ADM0_EN,ADM0_AR,ADM1_EN,ADM1_AR,ADM2_EN,ADM2_AR,geometry,rent/m,sale/m,house_counts_rent,house_counts_sale,rent/m(norm),sale/m(norm),index
0,Egypt,مِصر,Sharkia,الشرقية\n,10 Ramadan 1,قسم اول مدينة العاشر من رمض,"POLYGON ((31.70323 30.21166, 31.70354 30.21452...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Egypt,مِصر,Sharkia,الشرقية\n,10 Ramadan 2,قسم ثان مدينه العاشر من رمض,"POLYGON ((31.72752 30.25126, 31.72671 30.25009...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Egypt,مِصر,Cairo,القاهرة,15 Mayu,قسم 15 مايو,"POLYGON ((31.37739 29.78892, 31.36398 29.78987...",17.285814,3807.235259,19.0,30.0,0.042856,0.237343,0.140099
3,Egypt,مِصر,Giza,الجيزة,6 October-1,قسم أول 6 أكتوبر,"POLYGON ((30.98266 30.00870, 30.98959 30.00103...",39.461182,6994.312653,618.0,899.0,0.102899,0.446316,0.274608
4,Egypt,مِصر,Giza,الجيزة,6 October-2,قسم ثان 6 أكتوبر,"POLYGON ((30.95400 29.95111, 30.95393 29.95104...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,Egypt,مِصر,Luxor,مدينة الأقصر,Zemam Out,خارج الزمام,"POLYGON ((32.49782 25.54791, 32.49789 25.54802...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
361,Egypt,مِصر,Assiut,أسيوط,Zemam Out,صحراء أسيوط,"MULTIPOLYGON (((32.56866 27.85326, 32.56885 27...",28.359280,5381.562882,6.0,4.0,0.072839,0.340570,0.206704
362,Egypt,مِصر,Suhag,سوهاج,Zemam Out,صحراء سوهاج,"MULTIPOLYGON (((32.74119 26.68548, 32.28002 26...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
363,Egypt,مِصر,Qena,قنا,Zemam Out,صحراء قنا,"MULTIPOLYGON (((32.93116 26.69319, 32.93297 26...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


Exporting Final Data

In [ ]:
marakez_lev2.drop(["geometry"], axis = 1).to_csv('final.csv', index=False, encoding='utf-8-sig')

In [ ]:
from google.colab import files
files.download("final.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>